# Imports

## Importando bibliotecas

In [1]:
import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import darts  # Biblioteca para previsão de séries temporais
from darts.utils.statistics import check_seasonality, plot_acf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb



custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

In [2]:
#dia 1 (16/05/2024)
#d1_train = pd.read_csv('d1_train.csv')
#d1_test_hr = pd.read_csv('d1_teste_hora.csv')
#d1_test_local1= pd.read_csv('d1_local_1.csv')
#d1_test_local2= pd.read_csv('d1_local_2.csv')
#dia 2 (17/05/2024)
#d2_train = pd.read_csv('d2_train.csv')
#d2_test_hr = pd.read_csv('d2_test_hora.csv')
#d2_test_local1= pd.read_csv('d2_local_1.csv')
#d2_test_local2= pd.read_csv('d2_local_2.csv')
#dia 3 (18/05/2024)
#d3_train = pd.read_csv('d3_train.csv')
#d3_test_hr = pd.read_csv('d3_test_hora.csv')
#d3_test_local1= pd.read_csv('d3_local_1.csv')
#d3_test_local2= pd.read_csv('d3_local_2.csv')
#dia 4 (19/05/2024)
#d4_train = pd.read_csv('d4_train.csv')
#d4_test_hr = pd.read_csv('d4_test_hora.csv')
#d4_test_local1= pd.read_csv('d4_local_1.csv')
#d4_test_local2= pd.read_csv('d4_local_2.csv')
#dia 5 (20/05/2024)
d5_train = pd.read_csv('d5_train.csv')
d5_test_hr = pd.read_csv('d5_test_hora.csv')
d5_test_local1= pd.read_csv('d5_local_1.csv')
d5_test_local2= pd.read_csv('d5_local_2.csv')

Os `.csv` de treino já foram tratados utilizando SQL para filtrar os ônibus que estão parados na garagem

In [4]:
def summary(df):
    summ = pd.DataFrame(index = list(df))
    summ['type'] = df.dtypes
    summ['count'] = df.count()
    summ['nunique'] = df.nunique()
    summ['%unique'] = summ['nunique'] / len(df) * 100
    summ['null'] = df.isnull().sum()
    summ['%null'] = summ['null'] / len(df) * 100
    summ['min'] = df.min()
    summ['max'] = df.max()
    return summ

In [ ]:
summary(d5_train)

# Criando o Modelo para latitude e longitude

In [3]:
label_encoder_ordem = LabelEncoder()

d5_train['ordem_encoded'] = label_encoder_ordem.fit_transform(d5_train['ordem'])
d5_test_hr['ordem_encoded'] = label_encoder_ordem.fit_transform(d5_test_hr['ordem'])

## Latitude

In [4]:
features = [
    'linha',
    'datahora',
    'ordem_encoded',
]

target_lat = 'latitude'

X = d5_train[features]
y_lat = d5_train[target_lat]

X_train, X_test, y_train_lat, y_test_lat = train_test_split(X, y_lat, test_size=0.2, random_state=42)

In [ ]:
xgb_lat = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,          
    learning_rate=0.05,         
    max_depth=5,                
    subsample=0.8,              
    colsample_bytree=0.8,       
    random_state=42,
    n_jobs=-1                   
)

# Treinar o modelo
xgb_lat.fit(X_train, y_train_lat,
          eval_set=[(X_test, y_test_lat)], verbose=False)

# 4. Avaliar o modelo
preds_lat = xgb_lat.predict(X_test)
rmse_lat = np.sqrt(mean_squared_error(y_test_lat, preds_lat))
print(f"O RMSE para a previsão de Latitude é: {rmse_lat}")

## Longitude

In [6]:
target_lon = 'longitude'
y_lon = d5_train[target_lon]

# 2. Dividir os dados em treino e teste
X_train, X_test, y_train_lon, y_test_lon = train_test_split(X, y_lon, test_size=0.2, random_state=42)

In [ ]:
xgb_lon = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Treinar o modelo
xgb_lon.fit(X_train, y_train_lon,
          eval_set=[(X_test, y_test_lon)],
          verbose=False)

# 4. Avaliar o modelo
preds_lon = xgb_lon.predict(X_test)
rmse_lon = np.sqrt(mean_squared_error(y_test_lon, preds_lon))
print(f"O RMSE para a previsão de Longitude é: {rmse_lon}")

## Aplicando o modelo nos arquivos de teste

eu não tinha visto que os arquivos eram pra ser enviados separados por hora, tinha pensado que era so por tipo de previsão por isso que tive que colocar esse código para dividir os dataframes

In [8]:
d5_test_hr['datahora_dt'] = pd.to_datetime(d5_test_hr['datahora'], unit='ms')


horario_de_corte = pd.Timestamp('2024-05-20 17:00:00')


test_hr_1 = d5_test_hr[d5_test_hr['datahora_dt'] < horario_de_corte].copy()

test_hr_2 = d5_test_hr[d5_test_hr['datahora_dt'] >= horario_de_corte].copy()

In [9]:
id1 = test_hr_1['indx']
id2 = test_hr_2['indx']

df_prev1 = test_hr_1[features]
df_prev2 = test_hr_2[features]

In [10]:
pred1_lat = xgb_lat.predict(df_prev1)
pred1_lon = xgb_lon.predict(df_prev1)

pred2_lat = xgb_lat.predict(df_prev2)
pred2_lon = xgb_lon.predict(df_prev2)

In [14]:
#previsoes = [[i, float(la), float(lo)] for i, la, lo in zip(id1, pred1_lat, pred1_lon)]
previsoes = [[i, float(la), float(lo)] for i, la, lo in zip(id2, pred2_lat, pred2_lon)]

In [ ]:
print(json.dumps(previsoes, indent=2))

## Criando o arquivo `.json`

In [15]:
dados_para_submissao = {
    "aluno": "Luiz Guilherme de Andrade Pires",
    "senha": "VascodaGama",
    "datahora": "2024-05-20 17:00:00", 
    "previsoes": previsoes
}

In [16]:
file_path = 'resposta.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(dados_para_submissao, f, ensure_ascii=False, indent=4)

# Criando o modelo para encontrar a DataHora

In [17]:
features_hr = [
    'linha',
    'latitude',
    'longitude',
    'ordem_encoded',
]

target_hr = 'datahora'

In [18]:
X = d5_train[features_hr]
y_hr = d5_train[target_hr]

X_train, X_test, y_train_hr, y_test_hr = train_test_split(X, y_hr, test_size=0.2, random_state=42)

In [ ]:
xgb_hr = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Treinar o modelo
xgb_hr.fit(X_train, y_train_hr,
          eval_set=[(X_test, y_test_hr)],
          verbose=False)

# 4. Avaliar o modelo
preds_hr = xgb_hr.predict(X_test)
rmse_hr = np.sqrt(mean_squared_error(y_test_hr, preds_hr))
print(f"O RMSE para a previsão de datahora é: {rmse_hr}")

## Aplicando o modelo no teste

In [20]:
d5_test_local1['ordem_encoded'] = label_encoder_ordem.fit_transform(d5_test_local1['ordem'])
d5_test_local2['ordem_encoded'] = label_encoder_ordem.fit_transform(d5_test_local2['ordem'])

In [21]:
id1 = d5_test_local1['indx']
id2 = d5_test_local2['indx']

In [22]:
df_prev1_hr = d5_test_local1[features_hr]
df_prev2_hr = d5_test_local2[features_hr]

In [23]:
pred1_hr = xgb_hr.predict(df_prev1_hr)
pred2_hr = xgb_hr.predict(df_prev2_hr)

In [31]:
#previsoes_hr = [[i, int(hr)] for i, hr in zip(id1, pred1_hr)]
previsoes_hr = [[i, int(hr)] for i, hr in zip(id2, pred2_hr)]

In [ ]:
print(json.dumps(previsoes_hr, indent=2))

## Criando o arquivo `.json`

In [32]:
dados_para_submissao_hr = {
    "aluno": "Luiz Guilherme de Andrade Pires",
    "senha": "VascodaGama",
    "datahora": "2024-05-20 21:00:00", 
    "previsoes": previsoes_hr
}

In [33]:
file_path = 'resposta.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(dados_para_submissao_hr, f, ensure_ascii=False, indent=4)